# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import (g_key, weather_api_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data= "Output/city_data.csv"
weather_df= pd.read_csv(city_data)
weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,archidona,37.10,-4.39,288.71,83,0,0.89,ES,1603596520
1,batemans bay,-35.72,150.18,287.59,93,100,1.34,AU,1603596520
2,sankheda,22.17,73.58,298.15,65,5,1.50,IN,1603596520
3,hermanus,-34.42,19.23,288.71,85,56,4.62,ZA,1603596420
4,leningradskiy,69.38,178.42,268.49,91,0,4.42,RU,1603596418
...,...,...,...,...,...,...,...,...,...
609,edgewater,28.99,-80.90,298.15,88,75,4.30,US,1603596562
610,castro,-24.79,-50.01,289.38,93,91,1.51,BR,1603596398
611,novo oriente,-5.53,-40.77,298.72,61,27,1.43,BR,1603596562
612,atuona,-9.80,-139.03,299.07,75,30,9.19,PF,1603596522


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

locations= weather_df[["Lat", "Lng"]].astype(float)
weight= weather_df["Humidity"].astype(float)

In [8]:
fig = gmaps.figure()
markers = gmaps.heatmap_layer(locations, weights=weight, max_intensity=100, point_radius = 6)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/output?parameters"

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [16]:
preferred_cities=weather_df.loc[(weather_df["Max Temp"]<=295) & (weather_df["Cloudiness"]<=15) & (weather_df["Wind Speed"]<=10)]
preferred_cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,archidona,37.10,-4.39,288.71,83,0,0.89,ES,1603596520
4,leningradskiy,69.38,178.42,268.49,91,0,4.42,RU,1603596418
21,mizdah,31.45,12.98,287.30,68,0,1.72,LY,1603596521
27,lebu,-37.62,-73.65,282.10,81,0,1.47,CL,1603596522
29,big spring,32.25,-101.48,291.15,52,1,5.10,US,1603596522
...,...,...,...,...,...,...,...,...,...
576,chuy,-33.70,-53.46,289.75,97,8,2.42,UY,1603596151
586,nalut,30.33,10.85,291.55,32,6,4.27,LY,1603596560
600,chiredzi,-21.05,31.67,291.57,82,0,1.42,ZW,1603596543
601,guerrero negro,27.98,-114.06,292.35,84,0,6.34,MX,1603596561


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
